In [ ]:
# Plots for section 4.3 (mv and technology operation)

In [ ]:
# TODO
# Check how congestion rent should be calculated

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import math
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from datetime import date, datetime, time, timedelta

# imported own functions
from utils import market_values, market_values_by_time_index, nodal_balance, capacity, capacity_links, \
    capacity_storage_units, get_condense_sum, nodal_balance, generation, generation_links, \
    generation_storage_units, market_values_storage_units, market_values_links, time_stored_LIFO

# imported own definitions
from utils import carrier_colors, carrier_renaming, carrier_renaming_reverse
from utils import resistive_heater, gas_boiler, heat_pump,water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_el_gen_s, c_el_con_s

# general variables
font1 = {'fontname': 'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/4.3_systems_mv'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]
markers = ["v", "^", "<", ">", "1", "2", "3", "4", "*", "+", "d", "o", "|", "s", "P", "p", "h"]

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [ ]:
# stst and exp dataframes
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [ ]:
# Notebook Functions

def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g


In [ ]:
assert 0

AssertionError: 

## General (4.3.1)

#### MV across regions

In [ ]:
# Further investigation:

df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean().mean()
df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean().mean()

df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean() - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean()
(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean() - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean()) / df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean()

(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["75%"] - df_stst_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["25%"]) - (df_exp_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["75%"] - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["25%"])

(df_stst_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["std"] - df_exp_ons[[c + "_mv" for c in c_el_gen_s]].describe().loc["std"])

df_stst_ons[[c + "_mv" for c in c_el_gen_s]].mean().sort_values()
df_exp_ons[[c + "_mv" for c in c_el_gen_s]].mean().sort_values()

In [ ]:
# electricity generating technologies

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_gen = df_stst_ons[[c + "_mv_qt" for c in c_el_gen_s]]
stst_el_gen = stst_el_gen.values
exp_el_gen = df_exp_ons[[c + "_mv_qt" for c in c_el_gen_s]]
exp_el_gen = exp_el_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_gen)
filtered_stst = [d[m] for d, m in zip(stst_el_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_gen)
filtered_exp = [d[m] for d, m in zip(exp_el_gen.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_gen_s]
index = c_el_gen_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )


# generation weighted average
gwa_mv_stst =np.multiply(df_stst_ons[[c + "_mv_qt" for c in c_el_gen_s]], (df_stst_ons[[c + "_gen" for c in c_el_gen_s]] / df_stst_ons[[c + "_gen" for c in c_el_gen_s]].sum())).sum()
gwa_mv_exp =np.multiply(df_exp_ons[[c + "_mv_qt" for c in c_el_gen_s]], (df_exp_ons[[c + "_gen" for c in c_el_gen_s]] / df_exp_ons[[c + "_gen" for c in c_el_gen_s]].sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, gwa_mv_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, gwa_mv_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_mv_qt" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_mv_qt" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("$€/MWh_{el}$")
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_mv_qt" for c in c_el_gen_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_mv_qt" for c in c_el_gen_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'])

fig.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}/mv_el_gen_both_box.png")

In [ ]:
# check if np.multiply handles nans right
df = np.multiply(df_stst_ons[[c + "_mv" for c in c_el_gen_s]], (df_stst_ons[[c + "_gen" for c in c_el_gen_s]] / df_stst_ons[[c + "_gen" for c in c_el_gen_s]].sum())).sum()
for i, c in enumerate(c_el_gen_s):
    print((df_stst_ons[f"{c}_mv"] * (df_stst_ons[f"{c}_gen"] / df_stst_ons[f"{c}_gen"] .sum())).sum() == df[i])

In [ ]:
# electricity consuming technologies
c_el_con_s

In [ ]:
# exclude ones with less than 50 TWh consumption in both scenarios: home battery charger
#df_exp_ons["H2 Electrolysis_con_el"].sum() < 50*1e3

In [ ]:
# electricity consuming technologies

# boxplot properties
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_con = df_stst_ons[[c + "_cost_mv_el_qt" for c in c_el_con_s]]
stst_el_con = stst_el_con.values
exp_el_con = df_exp_ons[[c + "_cost_mv_el_qt" for c in c_el_con_s]]
exp_el_con = exp_el_con.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_con)
filtered_stst = [d[m] for d, m in zip(stst_el_con.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_con)
filtered_exp = [d[m] for d, m in zip(exp_el_con.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_con_s]
index = c_el_con_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )

# consumption weighted average
cwa_mv_stst =np.multiply(df_stst_ons[[c + "_cost_mv_el_qt" for c in c_el_con_s]], (abs(df_stst_ons[[c + "_con_el" for c in c_el_con_s]]) / abs(df_stst_ons[[c + "_con_el" for c in c_el_con_s]]).sum())).sum()
cwa_mv_exp =np.multiply(df_exp_ons[[c + "_cost_mv_el_qt" for c in c_el_con_s]], (abs(df_exp_ons[[c + "_con_el" for c in c_el_con_s]]) / abs(df_exp_ons[[c + "_con_el" for c in c_el_con_s]]).sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, cwa_mv_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, cwa_mv_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)

for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_cost_mv_el_qt" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_cost_mv_el_qt" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("$€/MWh_{el}$")
plt.xticks(rotation=90)
# plt.title("Market values of electricity consuming technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_cost_mv_el_qt" for c in c_el_con_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_cost_mv_el_qt" for c in c_el_con_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'], loc="upper left")

fig.tight_layout()
plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv_el_con_both_box.png")

In [ ]:
# Further investigation
df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().mean()
df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().mean()

df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean() - df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean()

(df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["75%"] -
 df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["25%"]) - (
            df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["75%"] -
            df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["25%"])

(df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["std"] -
 df_exp_ons[[c + "_cost_mv_el" for c in c_el_con_s]].describe().loc["std"])

df_stst_ons[[c + "_cost_mv_el" for c in c_el_con_s]].mean().sort_values()

In [ ]:
gwa_mv_stst

In [ ]:
gwa_mv_exp

In [ ]:
for c in c_el_con_s:
    print((df_stst_ons[f"{c}_cost_mv_el_qt"] * (abs(df_stst_ons[f"{c}_con_el"]) / abs(df_stst_ons[f"{c}_con_el"]).sum())).sum())


In [ ]:
df_exp_ons[f"{c}_con_el"]

## Spatial Differences (4.3.2)

### Spatial correlation of mv, gen

In [ ]:
# Spatial correlation of mv

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))

for i, df, ax in zip([0,1], [df_stst_ons, df_exp_ons], axs):

    corr = df[[c + "_mv" for c in c_el_gen_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_gen_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_gen_s]
    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.4], mask=mask, cmap="magma_r", annot=True, ax=ax)
    ax.set_title('STST' if i == 0 else 'EXP')

fig.tight_layout(pad=1.5)
plt.close()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv_el_gen_both_corr.png")

In [ ]:
# Spatial correlation of purchasing prices

fig, axs = plt.subplots(ncols=2, figsize=(8, 3))

for df, ax in zip([df_stst_ons, df_exp_ons], axs):
    corr = df[[c + "_cost_mv_el" for c in c_el_con_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_con_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_con_s]

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.5], mask=mask, cmap="magma_r", annot=True, ax=ax)
fig.tight_layout(pad=1.5)

plt.close()
plt.show()

In [ ]:
# Spatial correlation of both

fig, axs = plt.subplots(ncols=2, figsize=(17, 6))

for df, ax in zip([df_stst_ons, df_exp_ons], axs):
    corr = df[[c + "_mv" for c in c_el_gen_s] + [c + "_cost_mv_el" for c in c_el_con_s]].corr()
    corr.index = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n) + " (con)" for n in c_el_con_s]
    corr.columns = [carrier_renaming.get(n, n) for n in c_el_gen_s] + [carrier_renaming.get(n, n)+ " (con)" for n in c_el_con_s]

    mask = np.triu(np.ones_like(corr, dtype=bool))

    sns.heatmap(corr[abs(corr) > 0.5], mask=mask, cmap="magma_r", annot=True, ax=ax)
fig.tight_layout(pad=1.5)
plt.close()
plt.show()

In [ ]:
# correlation analysis
# what do you want to find?


corr_gini_res = pd.DataFrame(index=c_el_gen_s, columns=[["corr_mv_gen_STST", "cor_mv_gen_EXP", "gini_gen_STST", "gini_gen_EXP", "gini_gen_diff_EXP-STST", "gini_mv_STST", "gini_mv_EXP", "gini_mv_diff_EXP-STST"]])
for c in c_el_gen_s:
    corr_gini_res.loc[c,"corr_mv_gen_STST"] = df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"])
    corr_gini_res.loc[c,"cor_mv_gen_EXP"] = df_exp_ons[f"{c}_mv"].corr(df_exp_ons[f"{c}_gen"])
    corr_gini_res.loc[c,"gini_gen_STST"] = gini(df_stst_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_gen_EXP"] = gini(df_exp_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_gen_EXP"] = gini(df_exp_ons[[f"{c}_gen"]].dropna().values)
    corr_gini_res.loc[c,"gini_mv_STST"] = gini(df_stst_ons[[f"{c}_mv"]].dropna().values)
    corr_gini_res.loc[c,"gini_mv_EXP"] = gini(df_exp_ons[[f"{c}_mv"]].dropna().values)

corr_gini_res["gini_gen_diff_EXP-STST"] = corr_gini_res["gini_gen_EXP"].values - corr_gini_res["gini_gen_STST"].values
corr_gini_res["gini_mv_diff_EXP-STST"] = corr_gini_res["gini_mv_EXP"].values - corr_gini_res["gini_mv_STST"].values
corr_gini_res

**Findings:**
- gini: the higher, the more uneven distributed
- generation is more uneven distributed in EXP case
- mv are more evenly distributed in EXP case

### Electricity producing technologies

#### VRE

In [ ]:
# VRE: market values across regions as map
# ["onwind", "ror", "solar", "solar rooftop", "offwind-dc", "offwind-ac"]
carriers = ["onwind", "solar","offwind-dc", "ror"]
model = "STST"

fig, axs = plt.subplots(ncols=2, nrows=int(len(carriers)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, len(carriers)/2*8))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if model == "STST":
        if carriers[i] in ["offwind-dc", "offwind-ac"]:
            df = df_stst_off
        else:
            df = df_stst_ons

    elif model == "EXP":
        if carriers[i] in ["offwind-dc", "offwind-ac"]:
            df = df_exp_off
        else:
            df = df_exp_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_qt",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_gen_vre_map_{model}.png")

In [ ]:
# Further investigation
# TODO why is the correlation between mv and cp for ror positive
c_vre = ["onwind", "solar", "offwind-dc", "solar rooftop", "offwind-ac", "ror"]
for c in c_vre:
    print(c)
    print(df_stst_ons[f"{c}_mv_qt"].corr(df_stst_ons[f"{c}_gen"]))
    print(df_stst_ons[f"{c}_mv_qt"].corr(df_stst_ons[f"{c}_cf"]))

# generation weighted market value
c = "ror"
((df_stst_ons[f"{c}_gen"] / df_stst_ons[f"{c}_gen"].sum()) * df_stst_ons[f"{c}_mv_qt"]).sum()

In [ ]:
# Further investigation: EXP
# TODO why is the correlation between mv and cp for ror positive
c_vre = ["onwind", "solar", "offwind-dc", "solar rooftop", "offwind-ac", "ror"]
for c in c_vre:
    print(c)
    print(df_stst_ons[f"{c}_mv_qt"].corr(df_exp_ons[f"{c}_gen"]))
    print(df_stst_ons[f"{c}_mv_qt"].corr(df_exp_ons[f"{c}_cf"]))

# generation weighted market value
c = "ror"
((df_stst_ons[f"{c}_gen"] / df_stst_ons[f"{c}_gen"].sum()) * df_stst_ons[f"{c}_mv"]).sum()

In [ ]:
# VRE difference
# VRE: difference market values across regions as map
# ["onwind", "ror", "solar", "solar rooftop", "offwind-dc", "offwind-ac"]

carriers = ["onwind", "solar","offwind-dc", "ror"]

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_qt_STST-EXP"] = df_stst_off[f"{c}_mv_qt"] - df_exp_off[f"{c}_mv_qt"]
    else:
        df_stst_ons[f"{c}_mv_qt_STST-EXP"] = df_stst_ons[f"{c}_mv_qt"] - df_exp_ons[f"{c}_mv_qt"]

fig, axs = plt.subplots(ncols=2, nrows=int(len(carriers)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, len(carriers)/2*5))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_qt_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_qt_STST-EXP"].min()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_qt_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production in STST', 'Increased production in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]} mv (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_gen_vre_STST-EXP_map.png")

In [ ]:
# Further investigation
c = "onwind"
df_stst_ons[f"{c}_mv_qt_STST-EXP"].dropna().sort_values()

In [ ]:
de_i = df_stst_ons[f"{c}_mv_qt_STST-EXP"].dropna().index.str.contains("BE")
df_stst_ons[f"{c}_mv_qt_STST-EXP"].dropna().loc[de_i].mean()

#### Storage

In [ ]:
# Storage: market values across regions as map
carriers = ["hydro", "battery discharger",  "V2G", "PHS"]
model = "STST"

if model == "STST":
    df = df_stst_ons

elif model == "EXP":
    df = df_exp_ons

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_qt",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_gen_store_{model}_map.png")

In [ ]:
# Further investigate
c_el_gen_s

In [ ]:
c = "battery discharger"
df_stst_ons[F"{c}_mv_qt"].dropna().sort_values()

In [ ]:
# Storage difference

# market values across regions as map
carriers = [ "hydro", "battery discharger", "V2G", "PHS"]

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_qt_STST-EXP"] = df_stst_off[f"{c}_mv_qt"] - df_exp_off[f"{c}_mv_qt"]
    else:
        df_stst_ons[f"{c}_mv_qt_STST-EXP"] = df_stst_ons[f"{c}_mv_qt"] - df_exp_ons[f"{c}_mv_qt"]

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_qt_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_qt_STST-EXP"].min()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_qt_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production in STST', 'Increased production in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]} mv (STST - EXP)", fontsize=16, **font1)

fig.tight_layout()

#plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_gen_store_STST-EXP_map.png")

In [ ]:
# Further investigation
c = "hydro"
df_stst_ons[f"{c}_mv_qt_STST-EXP"].sort_values()

In [ ]:
df_exp_ons[[f"{c}_mv",f"{c}_gen"]].sort_values(by=f"{c}_mv")

In [ ]:
# difference in mv
pd.concat([df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"], df_stst_ons[f"{c}_gen"],df_exp_ons[f"{c}_gen"]], axis=1).sort_values(by=f"{c}_mv")

In [ ]:
for c in c_el_gen_s:
    print(c)
    mv_diff = df_stst_ons[f"{c}_mv"] - df_exp_ons[f"{c}_mv"]
    gen_diff = df_stst_ons[f"{c}_gen"] - df_exp_ons[f"{c}_gen"]
    print(mv_diff.corr(gen_diff))

In [ ]:
# Norway analysis
i_no = df_stst_ons.index[df_stst_ons.index.str.contains("NO")]
# generation weighted market value
c = "hydro"
((df_stst_ons.loc[i_no, f"{c}_gen"] / df_stst_ons.loc[i_no, f"{c}_gen"].sum()) * df_stst_ons.loc[i_no,f"{c}_mv"]).sum()
#df_stst_ons.loc[i_no, f"{c}_gen"].sum()

In [ ]:
c = "V2G"
((df_exp_ons.loc[i_no, f"{c}_gen"] / df_exp_ons.loc[i_no, f"{c}_gen"].sum()) * df_exp_ons.loc[i_no,f"{c}_mv"]).sum()
df_exp_ons.loc[i_no, f"{c}_gen"].sum()

In [ ]:
c_vre = ["hydro", "battery discharger", "V2G", "PHS"]
for c in c_vre:
    print(c)
    print(df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"]))
    print(df_stst_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_cf"]))

In [ ]:
c_vre = ["hydro", "battery discharger", "V2G", "PHS"]
for c in c_vre:
    print(c)
    print(df_exp_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_gen"]))
    print(df_exp_ons[f"{c}_mv"].corr(df_stst_ons[f"{c}_cf"]))

In [ ]:
# generation weighted market value
c = "hydro"
((df_stst_ons[f"{c}_gen"] / df_stst_ons[f"{c}_gen"].sum()) * df_stst_ons[f"{c}_mv"]).sum()

In [ ]:
# generation weighted market value
c = "hydro"
((df_exp_ons[f"{c}_gen"] / df_exp_ons[f"{c}_gen"].sum()) * df_exp_ons[f"{c}_mv"]).sum()

In [ ]:
# Storage: Battery Discharger

carriers = ["battery discharger", "battery discharger"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},figsize=(18, 8),
                        gridspec_kw={'width_ratios': [1, 0.96], 'height_ratios': [1]})
crs = ccrs.EqualEarth()

# same colorbar withds:
vmin = min(np.nanmin(df_stst_ons[f"{carriers[0]}_mv"]), np.nanmin(df_exp_ons[f"{carriers[0]}_mv"]))
vmax = max(np.nanmax(df_stst_ons[f"{carriers[0]}_mv"]), np.nanmax(df_exp_ons[f"{carriers[0]}_mv"]))

for i, ax in enumerate(axs):

    if i == 0:
        df = df_stst_ons
    else:
        df = df_exp_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   vmin=vmin,
                                   vmax=vmax,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    model = 'STST' if i == 0 else 'EXP'
    ax.set_title(f"{carriers[i]} ({model})", fontsize=16, **font1)

fig.tight_layout()

plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_gen_bat_disch_STST_EXP_map.png")

In [ ]:
# from here on with nem df_CALC / CALC

#### Peak plants

In [ ]:
# Peak plants

carriers = ["urban central gas CHP", "urban central solid biomass CHP CC"]
model = "STST"

if model == "STST":
    df = df_stst_ons

elif model == "EXP":
    df = df_exp_ons

fig, axs = plt.subplots(ncols=2, nrows=1,
                        subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 8))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_gen_el_qt",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market values ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen_el"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carrier_renaming[carriers[i]]}", fontsize=16, **font1)

fig.tight_layout()

#plt.close()
plt.show()


# fig.savefig(f"{PLOT_DIR}/mv_el_gen_peak_{model}_map.png")

In [ ]:
c = "urban central solid biomass CHP CC"
df_stst_ons[[f"{c}_mv_gen_el_qt",f"{c}_gen_el"]].dropna().sort_values(by=f"{c}_mv_gen_el_qt")
df_stst_ons[f"{c}_mv_gen_el_qt"][df_stst_ons[f"{c}_gen_el"] > 1000].dropna().sort_values()

In [ ]:
df_stst_ons

In [ ]:
# Storage difference

# market values across regions as map
carriers = ["urban central gas CHP", "urban central solid biomass CHP CC"]

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_gen_el_qt_STST-EXP"] = df_stst_off[f"{c}_mv_gen_el_qt"] - df_exp_off[f"{c}_mv_gen_el_qt"]
    else:
        df_stst_ons[f"{c}_mv_gen_el_qt_STST-EXP"] = df_stst_ons[f"{c}_mv_gen_el_qt"] - df_exp_ons[f"{c}_mv_gen_el_qt"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 8))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_gen_el_qt_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_gen_el_qt_STST-EXP"].max()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_gen_el_qt_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"market value differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.8}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen_el"] - df_2[f"{carriers[i]}_gen_el"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased production in STST', 'Increased production in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carrier_renaming[carriers[i]]} mv (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the electricity generation of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_gen_peak_STST-EXP_map.png")

In [ ]:
c = "urban central solid biomass CHP CC"
df_stst_ons[f"{c}_mv_gen_el_qt_STST-EXP"].sort_values().dropna().mean()

In [ ]:
stst.links[stst.links.carrier == "urban central solid biomass CHP CC"].head()

### Electricity Consuming Technologies

In [ ]:
df_stst_ons.columns[df_stst_ons.columns.str.contains("cap_con_el")].str.split('_').str[0]

In [ ]:
c_el_con_s

In [ ]:
carriers = c_el_con_s
model = "STST"

if model == "STST":
    df = df_stst_ons

elif model == "EXP":
    df = df_exp_ons

fig, axs = plt.subplots(ncols=2, nrows=int(len(carriers)/2),
                        subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, len(carriers)/2*6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_con_el_qt",
                                   ax=ax,
                                   cmap=plt.get_cmap("magma_r"),
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"Purchasing prices ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    max_size = abs(df[f"{carriers[i]}_con_el"].min())
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=abs(df[f"{carriers[i]}_con_el"]) / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Consumption magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]}", fontsize=14, **font1)

fig.tight_layout()

plt.close()
plt.show()


# fig.savefig(f"{PLOT_DIR}/mv_el_con_{model}_map.png")

In [ ]:
c = "urban central resistive heater"
df_stst_ons[[f"{c}_mv_con_el_qt", f"{c}_con_el"]].dropna().sort_values(by=f"{c}_mv_con_el_qt")
# corr
#df_stst_ons[f"{c}_mv_con_el_qt"].corr(abs(df_stst_ons[f"{c}_con_el"]))
#df_stst_ons[f"{c}_mv_con_el_qt"][abs(df_stst_ons[f"{c}_con_el"]) > 70 *1e3]#.mean()

In [ ]:
df_exp_ons[[f"{c}_mv_con_el_qt", f"{c}_con_el"]].dropna().sort_values(by=f"{c}_mv_con_el_qt")

In [ ]:

df_exp_ons[[f"{c}_mv_con_el_qt", f"{c}_con_el"]].dropna().sort_values(by=f"{c}_mv_con_el_qt")


In [ ]:
c = "H2 Electrolysis"
df_stst_ons[[f"{c}_mv_con_el_qt",f"{c}_cost_mv_el_qt"]]

In [ ]:
# consumption weighted market value
c = "battery charger"
((abs(df_stst_ons[f"{c}_con_el"]) / abs(df_stst_ons[f"{c}_gen"]).sum()) * df_stst_ons[f"{c}_mv_gen_el_qt"]).sum()
((abs(df_exp_ons[f"{c}_con_el"]) / abs(df_exp_ons[f"{c}_gen"]).sum()) * df_exp_ons[f"{c}_mv_gen_el_qt"]).sum()

In [ ]:
# Electricity consuming difference

# market values across regions as map
carriers = c_el_con_s

for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_mv_con_el_qt_STST-EXP"] = df_stst_off[f"{c}_mv_con_el_qt"] - df_exp_off[f"{c}_mv_con_el_qt"]
    else:
        df_stst_ons[f"{c}_mv_con_el_qt_STST-EXP"] = df_stst_ons[f"{c}_mv_con_el_qt"] - df_exp_ons[f"{c}_mv_con_el_qt"]

fig, axs = plt.subplots(ncols=2, nrows=int(len(carriers)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(14, len(carriers)/2*6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons

    abs_max = max(abs(df[f"{carriers[i]}_mv_con_el_qt_STST-EXP"].max()) , abs(df[f"{carriers[i]}_mv_con_el_qt_STST-EXP"].max()))

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_mv_con_el_qt_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':"Purchasing prices differences ($€/MWh_{el}$)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    # difference in consumption (STST-EXP)
    con_diff = abs(df[f"{carriers[i]}_con_el"]) - abs(df_2[f"{carriers[i]}_con_el"])
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in con_diff ]

    max_size = abs(con_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(con_diff) / max_size) * 500,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased consumption in STST', 'Increased consumption in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carriers[i]} (STST - EXP)", fontsize=14, **font1)

# fig.suptitle("Spatial Differences in the electricity consumption of the VRE technologies (STST)", fontsize=16, **font1)
fig.tight_layout()

plt.close()
plt.show()


#fig.savefig(f"{PLOT_DIR}/mv_el_con_STST-EXP_map.png")

In [ ]:
c = "urban central air heat pump"
i_it = df_stst_ons.index.str.contains("IT")
df_stst_ons[f"{c}_mv_con_el_qt_STST-EXP"].sort_values()
df_stst_ons[f"{c}_mv_con_el_qt_STST-EXP"][i_it].mean()

### Value Factor Analysis

In [ ]:
df_stst_ons.columns[df_stst_ons.columns.str.contains("lmp")]


In [ ]:
df_stst_ons[[f"{c}_vf_gen_el" for c in c_el_gen_s]]

In [ ]:
# electricity consuming technologies

# boxplot properties
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_con = df_stst_ons[[c + "_vf_con_el_qt" for c in c_el_con_s]]
stst_el_con = stst_el_con.values
exp_el_con = df_exp_ons[[c + "_vf_con_el_qt" for c in c_el_con_s]]
exp_el_con = exp_el_con.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_con)
filtered_stst = [d[m] for d, m in zip(stst_el_con.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_con)
filtered_exp = [d[m] for d, m in zip(exp_el_con.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_con_s]
index = c_el_con_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )

# consumption weighted average
cwa_vf_stst =np.multiply(df_stst_ons[[c + "_vf_con_el_qt" for c in c_el_con_s]], (abs(df_stst_ons[[c + "_con_el" for c in c_el_con_s]]) / abs(df_stst_ons[[c + "_con_el" for c in c_el_con_s]]).sum())).sum()
cwa_vf_exp =np.multiply(df_exp_ons[[c + "_vf_con_el_qt" for c in c_el_con_s]], (abs(df_exp_ons[[c + "_con_el" for c in c_el_con_s]]) / abs(df_exp_ons[[c + "_con_el" for c in c_el_con_s]]).sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, cwa_vf_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, cwa_vf_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)

for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_vf_con_el_qt" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.4)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_vf_con_el_qt" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("")
plt.xticks(rotation=90)
# plt.title("Market values of electricity consuming technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_vf_con_el_qt" for c in c_el_con_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_vf_con_el_qt" for c in c_el_con_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'], loc="upper left")

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/vf_el_con_both_box.png")

In [ ]:
# electricity generating technologies

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_gen = df_stst_ons[[f"{c}_vf_gen_el_qt" for c in c_el_gen_s]]
stst_el_gen = stst_el_gen.values
exp_el_gen = df_exp_ons[[f"{c}_vf_gen_el_qt" for c in c_el_gen_s]]
exp_el_gen = exp_el_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_gen)
filtered_stst = [d[m] for d, m in zip(stst_el_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_gen)
filtered_exp = [d[m] for d, m in zip(exp_el_gen.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_gen_s]
index = c_el_gen_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )


# generation weighted average
gwa_vf_stst =np.multiply(df_stst_ons[[c + "_vf_gen_el_qt" for c in c_el_gen_s]], (df_stst_ons[[c + "_gen_el" for c in c_el_gen_s]] / df_stst_ons[[c + "_gen_el" for c in c_el_gen_s]].sum())).sum()
gwa_vf_exp =np.multiply(df_exp_ons[[c + "_vf_gen_el_qt" for c in c_el_gen_s]], (df_exp_ons[[c + "_gen_el" for c in c_el_gen_s]] / df_exp_ons[[c + "_gen_el" for c in c_el_gen_s]].sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, gwa_vf_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, gwa_vf_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_vf_gen_el_qt" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_vf_gen_el_qt" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("")
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_vf_gen_el_qt" for c in c_el_gen_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_vf_gen_el_qt" for c in c_el_gen_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'])
ax.set_ylim([0,4])

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/vf_el_gen_both_box.png")

In [ ]:
# electricity generating technologies

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_gen = df_stst_ons[[f"{c}_mv-lmp_gen_el_qt" for c in c_el_gen_s]]
stst_el_gen = stst_el_gen.values
exp_el_gen = df_exp_ons[[f"{c}_mv-lmp_gen_el_qt" for c in c_el_gen_s]]
exp_el_gen = exp_el_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_gen)
filtered_stst = [d[m] for d, m in zip(stst_el_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_gen)
filtered_exp = [d[m] for d, m in zip(exp_el_gen.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_gen_s]
index = c_el_gen_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )


# generation weighted average
gwa_mv_lmp_stst =np.multiply(df_stst_ons[[c + "_mv-lmp_gen_el_qt" for c in c_el_gen_s]], (df_stst_ons[[c + "_gen_el" for c in c_el_gen_s]] / df_stst_ons[[c + "_gen_el" for c in c_el_gen_s]].sum())).sum()
gwa_mv_lmp_exp =np.multiply(df_exp_ons[[c + "_mv-lmp_gen_el_qt" for c in c_el_gen_s]], (df_exp_ons[[c + "_gen_el" for c in c_el_gen_s]] / df_exp_ons[[c + "_gen_el" for c in c_el_gen_s]].sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, gwa_mv_lmp_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, gwa_mv_lmp_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_mv-lmp_gen_el_qt" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_mv-lmp_gen_el_qt" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("")
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_mv-lmp_gen_el_qt" for c in c_el_gen_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_mv-lmp_gen_el_qt" for c in c_el_gen_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'])
#ax.set_ylim([0,4])

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv-lmp_el_gen_both_box.png")

In [ ]:
# electricity consuming technologies

# boxplot properties
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_con = df_stst_ons[[c + "_mv-lmp_con_el_qt" for c in c_el_con_s]]
stst_el_con = stst_el_con.values
exp_el_con = df_exp_ons[[c + "_mv-lmp_con_el_qt" for c in c_el_con_s]]
exp_el_con = exp_el_con.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_con)
filtered_stst = [d[m] for d, m in zip(stst_el_con.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_con)
filtered_exp = [d[m] for d, m in zip(exp_el_con.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_con_s]
index = c_el_con_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops
                       )

# consumption weighted average
cwa_mv_lmp_stst =np.multiply(df_stst_ons[[c + "_mv-lmp_con_el_qt" for c in c_el_con_s]], (abs(df_stst_ons[[c + "_con_el" for c in c_el_con_s]]) / abs(df_stst_ons[[c + "_con_el" for c in c_el_con_s]]).sum())).sum()
cwa_mv_lmp_exp =np.multiply(df_exp_ons[[c + "_mv-lmp_con_el_qt" for c in c_el_con_s]], (abs(df_exp_ons[[c + "_con_el" for c in c_el_con_s]]) / abs(df_exp_ons[[c + "_con_el" for c in c_el_con_s]]).sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, cwa_mv_lmp_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, cwa_mv_lmp_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)

for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_mv-lmp_con_el_qt" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.4)/len(index),1), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_mv-lmp_con_el_qt" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.ylabel("")
plt.xticks(rotation=90)
# plt.title("Market values of electricity consuming technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)

# horizontal lines
ax.axhline(df_stst_ons[[c + "_mv-lmp_con_el_qt" for c in c_el_con_s]].mean().mean(), ls="--", c='black',linewidth=0.5)
ax.axhline(df_exp_ons[[c + "_mv-lmp_con_el_qt" for c in c_el_con_s]].mean().mean(), ls="-", c='black', linewidth=0.5)

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
line1 = Line2D([0], [0], ls="--", c='black', label="mean (STST)", linewidth=0.5)
line2 = Line2D([0], [0], ls="-", c='black', label="mean (EXP)", linewidth=0.5)
ax.legend([patch1, patch2, line1, line2], ['STST', 'EXP', 'mean (STST)','mean (EXP)'], loc="upper left")

fig.tight_layout()
plt.show()

#fig.savefig(f"{PLOT_DIR}/mv-lmp_el_con_both_box.png")

### Congestion rent

## Temporal Differences (4.3.3)